In [ ]:
''' breakdown by cell
    hopefully the last piece!! 
    
    cribbing a lot of code from make_validation_tbl_with_gvcf_data.ipynb '''

In [24]:
import pandas as pd

In [25]:
# make sure to grab the ones with the 'leftover' genes
    # dont care about gvcfs for this one
cereb_vcf = pd.read_csv('cereb_vcf_plus_additions.csv', index_col=0)

In [26]:
# subset to just non-syno muts
keep_cols_vcf = []
for x in list(cereb_vcf.columns):
    mutstr = x.split('_')[1]
    if mutstr[0] != mutstr[-1]:
        keep_cols_vcf.append(x)
        
cereb_vcf = cereb_vcf[keep_cols_vcf]

In [27]:
# now lets subset to just tumor cells 
tumor_cells_df = pd.read_csv('/Users/lincoln.harris/Desktop/laud_process_new_cells/sample_cell_key.csv', index_col=0)
tumor_cells_list = list(tumor_cells_df.cell_id)

In [28]:
cereb_vcf = cereb_vcf[cereb_vcf.index.isin(tumor_cells_list)]
print(cereb_vcf.shape)

(2998, 878)


In [29]:
# this is all i care about!
cereb_vcf

,AKT1_D323Y,AKT1_E17K,AKT1_G394W,AKT1_M306L,AKT1_R465C,ALK_K1431T,ALK_K1491R,ALK_L1318M,BAP1_A140V,BAP1_D34fs*35,...,VHL_G212*,BRIP1_E718V,BRIP1_G652E,BRIP1_K242N,CD74_R10W,NKX2-1_G134S,NKX2-1_G142R,NKX2-1_P155fs*43,NKX2-1_S79L,NKX2-1_T8M
sample,,,,,,,,,,,,,,,,,,,,,
A10_1001000407,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_1001000408,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_1001000412,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_B001544,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_B003518,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_B003523,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_B003527,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_B003529,0:0,0:0,0:0,0:0,0:0,0:0,4:4,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
A10_B003588,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,...,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0


In [95]:
# maybe the easiest way to start is by generating a by-sample list of all the muts
    # start with empty dict
cell_d = {}
for cell in list(cereb_vcf.index):
    cell_d[cell] = []

In [96]:
# fill in dict 
for cell in list(cereb_vcf.index):
    curr_muts = []
    for mut in list(cereb_vcf.columns):
        curr = cereb_vcf.loc[cell][mut]
        if curr !='0:0' and '*' not in mut:
            curr_muts.append(mut)
            
    cell_d[cell] = curr_muts

In [97]:
cell_d

{'A10_1001000407': ['EGFR_R521K'],
 'A10_1001000408': ['TP53_S241F'],
 'A10_1001000412': ['EGFR_R521K', 'TP53_S241F'],
 'A10_B001544': ['TPR_Q2287H'],
 'A10_B003518': [],
 'A10_B003523': [],
 'A10_B003527': [],
 'A10_B003529': ['ALK_K1491R', 'MAP2K2_R299M'],
 'A10_B003588': [],
 'A11_1001000407': ['EGFR_R521K', 'TP53_S241F'],
 'A11_1001000412': ['EGFR_R521K', 'TP53_S241F'],
 'A11_B002077': ['NKX2-1_G142R'],
 'A11_B002079': [],
 'A11_B003119': ['SMARCA4_R521W'],
 'A11_B003522': ['NFE2L2_K506N'],
 'A12_1001000317': ['HIF1A_A588T'],
 'A12_1001000407': ['EGFR_R521K', 'NFE2L2_G31E'],
 'A12_1001000412': ['EGFR_R521K'],
 'A12_B002077': ['KEAP1_M110I', 'KEAP1_R116W', 'NFE2L2_G81S'],
 'A12_B002078': ['KEAP1_D389Y'],
 'A12_B003116': [],
 'A12_B003518': ['KEAP1_G364D'],
 'A12_B003527': ['TP63_V364I'],
 'A12_B003529': [],
 'A13_1001000407': ['EGFR_L858R', 'TP53_S241F'],
 'A13_1001000408': ['TP53_S241F'],
 'A13_1001000412': [],
 'A13_B003112': ['TPR_M1L'],
 'A13_B003518': ['TPM3_V10M'],
 'A13_B0035

In [33]:
#///////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////

In [56]:
# need to add in fusions
fusions_x_cell = pd.read_csv('/Users/lincoln.harris/Desktop/laud_process_new_cells/fusions_nonimmune_cells_3.21.20.csv', index_col=0)

In [57]:
# lets start by limiting things to these 3754
fusions_x_cell_trim = fusions_x_cell[fusions_x_cell.index.isin(tumor_cells_list)]
fusions_x_cell_trim

,fusions
G11_B001475,"['MDM2--SOX5', 'TOMM22--EPCAM', 'EPS15L1--TRIM..."
O12_B003518,"['AC009961.3--BAZ2B', 'FAM83A--YARS', 'NBAS--A..."
J13_B003522,"['RP11-332H14.2--NEAT1', 'PRPF6--NDRG1', 'SPID..."
K20_B003527,"['RHEB--RPS27A', 'ZNF12--PHTF2', 'SRPRB--LAPTM..."
D10_B001546,"['DGKH--EPSTI1', 'REL--MTDH']"
M1_B003132,"['CFLAR--CEP72', 'BSG--CTBP2', 'CDH1--RGS16', ..."
H9_1001000408,"['ATP5C1--RPL8', 'ERCC3--NCOA6', 'MYO1E--ZCCHC..."
L9_1001000412,"['MTF2--PSMB2', 'MUC1--MEF2D', 'LAMB3--MUC1', ..."
C22_1001000408,"['MUC4--COX7A2L', 'PVRL2--ERCC1', 'MED8--BCL2L..."
A18_1001000408,"['ASS1--BANF1', 'PSMD12--TOR4A', 'HOXC6--RP11-..."


In [58]:
# lets hunt for just ALK
alk_eml4_cells = []
for cell in fusions_x_cell_trim.index:
    curr_fus = fusions_x_cell_trim.loc[cell].fusions
    
    for elm in curr_fus.split(','):
        if 'EML4--ALK' in elm or 'ALK--EML4' in elm:
            print(cell)
            alk_eml4_cells.append(cell)

M12_B003522
C2_B000862
E7_1001000317
G8_1001000317
M21_B003518
M11_B003522
J15_B000862
M2_B001460
B11_10001000325
G2_B000862
A7_10001000325
G1_1001000317


In [59]:
# and now ROS1
ros1_cd74_cells = []
ros1_ezr_cells = []

for cell in fusions_x_cell_trim.index:
    curr_fus = fusions_x_cell_trim.loc[cell].fusions
    
    for elm in curr_fus.split(','):
        if 'CD74--ROS1' in elm or 'ROS1--CD74' in elm:
            ros1_cd74_cells.append(cell)
        elif 'EZR--ROS1' in elm or 'ROS1--EZR' in elm:
            ros1_ezr_cells.append(cell)

In [98]:
# lets update the dictionary to include fusions
for cell in alk_eml4_cells:
    if cell in list(cell_d.keys()):
        curr = cell_d[cell]
        curr.append('ALK-EML4_fusion')   # think this is all we need to do 
    else:
        cell_d[cell] = ['ALK--EML4_fusion']  

In [99]:
# nice! 
cell_d['M12_B003522']
cell_d['G8_1001000317']
cell_d['J15_B000862']

['ALK_K1491R', 'ALK-EML4_fusion']

In [100]:
# now do the same for the other two driver fusions 
    # this looks good! 
for cell in ros1_cd74_cells:
    if cell in list(cell_d.keys()):
        curr = cell_d[cell]
        curr.append('ROS1--CD74_fusion') 
    else:
        cell_d[cell] = ['ROS1--CD74_fusion']  
        
        
for cell in ros1_ezr_cells:
    if cell in list(cell_d.keys()):
        curr = cell_d[cell]
        curr.append('ROS1--EZR_fusion') 
    else:
        cell_d[cell] = ['ROS1--EZR_fusion']  

In [106]:
ros1_ezr_cells
cell_d['G16_B003105']

['ROS1_K2228Q', 'ROS1_S2229C', 'ROS1--EZR_fusion']

In [107]:
#///////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////

In [112]:
# want to add in the rest of the tumor cells, even though they dont have muts
not_included = set(tumor_cells_list) - set(x_cell_df.index)
len(not_included)

755

In [ ]:
# add them straight to the dict
for x in not_included:
    cell_d[x] = []

In [144]:
# now make some sort of by-cell list
    # sort of gonna do this manually here
x_cell_df = pd.DataFrame(index=list(cell_d.keys()), columns=['mutations'])

# for loop it up!!
for key in list(cell_d.keys()):
    x_cell_df.loc[key].mutations = cell_d[key]

x_cell_df   # nice!!!

,mutations
A10_1001000407,[EGFR_R521K]
A10_1001000408,[TP53_S241F]
A10_1001000412,"[EGFR_R521K, TP53_S241F]"
A10_B001544,[TPR_Q2287H]
A10_B003518,[]
A10_B003523,[]
A10_B003527,[]
A10_B003529,"[ALK_K1491R, MAP2K2_R299M]"
A10_B003588,[]
A11_1001000407,"[EGFR_R521K, TP53_S241F]"


In [146]:
x_cell_df = x_cell_df.sort_index()
x_cell_df

,mutations
A10_1001000407,[EGFR_R521K]
A10_1001000408,[TP53_S241F]
A10_1001000412,"[EGFR_R521K, TP53_S241F]"
A10_B001544,[TPR_Q2287H]
A10_B003518,[]
A10_B003523,[]
A10_B003527,[]
A10_B003529,"[ALK_K1491R, MAP2K2_R299M]"
A10_B003588,[]
A11_1001000407,"[EGFR_R521K, TP53_S241F]"


In [147]:
x_cell_df.to_csv('mutations_x_cell_4.2.20.csv')